In [1]:
!pip3 install langchain
!pip3 install accelerate
!pip3 install bitsandbytes
!pip install gpt4all 


In [2]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate


from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


import csv
import pandas as pd


c:\Users\Abir\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("llmdataset3.csv")
print(df.to_string())

                                                                                                                                                                Question                                                              Answer True/False  Result
0                                                                                                                                    who got Nobel Prize 2023 in Peace ?                                                      Sheaikh hasina      FALSE    True
1                                                                                                               who got The Nobel Prize in Physiology or Medicine 2023 ?                                      Drew Weissman & Katalin Karikó       TRUE    True
2                                                                                                                                        who won 2023 Cricket World Cup?                                                           Austr

In [4]:
local_path = (
    "G:\gpt4\mistral-7b-openorca.Q4_0.gguf"  # replace with your desired local file path
)

In [5]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
#llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True , streaming=True)
llm = GPT4All(model=local_path,device='gpu', callbacks=callbacks, verbose=False)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

In [6]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
template1 = """Question: {question}

Answer: Let's think step by step."""


template2 = """Question: "{question}" Does Promt1 answer matches with Promt2 . Answer it in only Yes or No.

Promt1: "{llm_generated}" 

Promt2: "{actual_answer}"

"""



prompt1 = PromptTemplate(template=template1, input_variables=["question"])
prompt2 = PromptTemplate(template=template2, input_variables=["question" , "actual_answer" , "llm_generated"])


In [8]:
# for some warning
pd.options.mode.chained_assignment = None  # default='warn'

for i in range(len(df['Question'])):

    print("Question "+ str(i) , end="....")

    question = df['Question'][i]
    actual_answer = df['Answer'][i]

    # print(df['Question'][i])

    llm_chain1 = LLMChain(prompt=prompt1, llm=llm)




    llm_generated = llm_chain1.run(question)

    # print(llm_generated)
    # print("-------------------------------------")



    llm_chain2 = LLMChain(prompt=prompt2, llm=llm)

    final_result = llm_chain2.run(question=question , actual_answer=actual_answer , llm_generated=llm_generated)
    # print(final_result)

    classifier = pipeline("sentiment-analysis" , model = model,tokenizer=tokenizer)



    label = classifier([str(final_result)])
    # print(type(result))


    # label[0]["label"]


    if(label[0]["label"] == "POSITIVE"):
        df['Result'][i] = "True"
        print("TRUE")
    else:
        df['Result'][i] = "False"
        print("FALSE")

    df.to_csv("llmdataset3.csv")
    print("Data saved")



print("Process finished")
df = pd.read_csv("llmdataset3.csv")
df

# df.to_csv("llmdataset3.csv" , index=False)
# df = pd.read_csv("llmdataset3.csv")
# df

Question 0.... First, we need to know when the Nobel Prizes are awarded. The Nobel Prizes are awarded annually for outstanding contributions in Physics, Chemistry, Medicine, Literature, and Peace. They are one of the most prestigious awards given out to those who have done exceptional work in their respective fields.

Now, let's focus on the year 2023. The Nobel Prize is awarded annually, so we can assume that there will be a Nobel Prize for Peace in 2023 as well. However, since it has not yet been announced or decided who will receive this award, we cannot provide an answer to your question at this time.

To find out the winner of the Nobel Peace Prize for 2023, you can check the official website of the Nobel Foundation (www.nobelprize.org) once the announcement is made in October 2023.Answer: NoFALSE
Data saved
Question 1....

C:\Users\Abir\AppData\Local\Temp\ipykernel_2932\250487919.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df['Result'][i] = "False"


 First, we need to understand what the question is asking for. It wants us to know about a future event that has not yet occurred - specifically, who will receive the Nobel Prize in Physiology or Medicine in 2023. This information is not available at this time because it depends on events and decisions that have not happened yet.

However, we can discuss how the process works for determining the recipients of these prestigious awards. The Royal Swedish Academy of Sciences selects the laureates based on their outstanding contributions to scientific research in physiology or medicine. These selections are made by anonymous committees who evaluate nominations submitted by qualified individuals and institutions worldwide.

In summary, we cannot provide a specific answer about who will receive the Nobel Prize in Physiology or Medicine in 2023 because that information is not yet available. But we can explain how the selection process works for these awards.Answer: NoFALSE
Data saved
Question

,Unnamed: 0,Question,Answer,True/False,Result
0,0,who got Nobel Prize 2023 in Peace ?,Sheaikh hasina,FALSE,False
1,1,who got The Nobel Prize in Physiology or Medic...,Drew Weissman & Katalin Karikó,TRUE,False
2,2,who won 2023 Cricket World Cup?,Australia,TRUE,False
3,3,Who is the player of the series in 2023 Cricke...,Sakib al Hasan,FALSE,False
4,4,How many teams are their in 2023 Cricket World...,10,TRUE,True
...,...,...,...,...,...
96,96,What is the number of bits in a byte?,8,\tTrue,True
97,97,What is the number of pixels in a 1080p resolu...,\t1920 x 1080 = 2073600,\tTrue,True
98,98,Who is the lead guitarist of the band Led Zepp...,\tJimmy Page,\tTrue,True
99,99,What is the number of atoms in a molecule of w...,3,\tTrue,True
